# **Nivell 1**  

# **Cargamos los datos en Power BI**

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

try:
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='Data75000',
        database='sales'
    )
    
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        
        cursor = connection.cursor()
        
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print("You're connected to database: ", record[0])
        
        cursor.execute("SHOW TABLES")
        tablas = cursor.fetchall()
        print("Tablas disponibles:", tablas)
        
        tablas_dict = {}  # Diccionario para almacenar DataFrames
        for (tabla,) in tablas:  # Cada elemento en 'tablas' es una tupla
            print(f"Cargando datos de la tabla: {tabla}")
            query = f"SELECT * FROM {tabla}"
            
            cursor.execute(query)
            rows = cursor.fetchall()
            column_names = [desc[0] for desc in cursor.description]
            
            tablas_dict[tabla] = pd.DataFrame(rows, columns=column_names)
        
        print("Datos cargados correctamente en DataFrames.")
        
except Error as e:
    print("Error while connecting to MySQL:", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

# Obtener información y crear DataFrames individuales a partir de tablas_dict
for tabla_nombre, tabla_df in tablas_dict.items():
    globals()[tabla_nombre] = tabla_df
    
del tabla_df

transactions["amount"] = transactions["amount"].astype(float)
transactions['timestamp'] = pd.to_datetime(transactions['timestamp'])

# en "users" convertimos 'birth_date' a datetime
users["birth_date"] = pd.to_datetime(users["birth_date"], format='%b %d, %Y')

# calculo de edad 
from datetime import date
def calculate_age(birthdate):
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age
    
# añadimos un campo nuevo 'age'
users['age'] = users['birth_date'].apply(calculate_age)
tablas_dict['users'] = users

# en "products" campos "price" y "weight" son de tipo "object" y además "price" tiene el simbolo $
products["price"] = products["price"].astype(str)
products["price"] = products["price"].str.replace(r"[$]", "", regex=True).astype(float)
products["weight"] = products["weight"].astype(float)


# Merge transactions with companies
merged_df = pd.merge(
    tablas_dict['transactions'], 
    tablas_dict['companies'], 
    left_on='business_id', 
    right_on='company_id', 
    how='inner', 
    suffixes=('_transaction', '_company')
)

# Merge the resulting merged_df with users
merged_df = pd.merge(
    merged_df, 
    tablas_dict['users'], 
    left_on='user_id', 
    right_on='id',     
    how='inner', 
    suffixes=('', '_user'))

# Añadimos campos
merged_df['year_month'] = merged_df['timestamp'].dt.to_period('M').astype(str)
merged_df['year'] = merged_df['timestamp'].dt.year
merged_df['month'] = merged_df['timestamp'].dt.month
merged_df['declined_status'] = merged_df['declined'].map({0: 'Accepted', 1: 'Declined'})
merged_df["user_id"] = pd.to_numeric(merged_df["user_id"], errors='coerce').astype('Int64')
merged_df=merged_df.drop('id_user', axis=1)


# Ej 1 

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 

# dataset = pandas.DataFrame(amount)
# dataset = dataset.drop_duplicates()
merged_df=dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
sns.set(style="white")
sns.set_palette("pastel")
plt.figure(figsize=(6, 4))
bins = 8 
# Marcas en el eje X ajustamos a valores de los bins
    # plt.hist(data, bins=8) devuelve tres objetos:
    # frequencies (índice [0]): contiene la cantidad de observaciones en cada uno de los bins o intervalos
    # bin_edges (índice [1]): los bordes de los bins
    # patches (índice [2]): los objetos visualizados como las barras del histograma
bin_edges = plt.hist(merged_df['amount'], bins=bins, alpha=0.7)[1]           
bin_edges_int = np.round(bin_edges,-1)
plt.title('Distribution of Transaction Amounts', fontsize=16)
plt.xlabel('Amount of 1 transantion (€)')
plt.ylabel('Frequency')
plt.xticks(bin_edges_int)
plt.grid(axis='y', linestyle='--', alpha=0.5)                                                      

mean_amount = np.round(merged_df['amount'].mean())
plt.axvline(mean_amount, color='darkgreen', linestyle='--', linewidth=2, label=f'Promedio: {mean_amount:.0f} €')
plt.legend(loc='upper center')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout(pad=0)
plt.show()



# Ej 2

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 

# dataset = dataset.drop_duplicates()
# Paste or type your script code here:
merged_df = dataset
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")
sns.set_palette("pastel")
plt.figure(figsize=(7, 4))
sns.boxplot(data=merged_df, x='age', y='amount')
plt.title('Relation between age and amount', fontsize=16)
plt.xlabel('age')
plt.ylabel('amount (€)')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout(pad=0)
plt.show()

# Ej 3

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 

# dataset = pandas.DataFrame(country, declined_status)
# dataset = dataset.drop_duplicates()
merged_df=dataset
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set(style="white")
sns.set_palette("pastel")
plt.figure(figsize=(4,4))
total_amount = merged_df['amount'].sum()
df_pie=merged_df.groupby([ 'company_name'])['amount'].sum().reset_index()
df_pie['category'] = np.where(df_pie['amount'] < 0.05 * total_amount, 'Others*', None)
df_pie['company_name'] = np.where(df_pie['category'] == 'Others*', 'Others*', df_pie['company_name'])
df_pie = df_pie.groupby('company_name')['amount'].sum().reset_index()
df_pie=df_pie.sort_values(by='amount', ascending=True)
plt.pie(df_pie['amount'], labels=df_pie['company_name'], autopct='%1.0f%%',startangle=-40)
plt.title('Transaction Distribution by Company (€)', fontsize=16)
plt.text(0, -1.3, '* Others - companies with less than 5% of the total transactions', 
         fontsize=10, ha='center', color='gray')
plt.show()


# Ej 4

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 

# dataset = pandas.DataFrame(year, month, amount)
# dataset = dataset.drop_duplicates()
merged_df=dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.ticker import FuncFormatter


sns.set(style="white")
sns.set_palette("pastel")
# PARTE 1 - GRAFICO DE BARRAS
monthly_sales = merged_df.groupby('year_month')['amount'].sum()
plt.figure(figsize=(7, 4))
monthly_sales.plot(kind='bar')

# PARTE 2 OPCIONAL - LINEA TREND
import numpy as np
from sklearn.linear_model import LinearRegression 

# Convertimos las etiquetas de 'year_month' a valores numéricos para la regresión
numeric_time = np.arange(len(monthly_sales)) # Un arreglo unidimensional (1D) 
# print(numeric_time)
# [ 0  1  2  3  4  5  6  7  8  9 10 11 12]

# Ajustamos un modelo de regresión lineal
# Usamos LinearRegression de scikit-learn para encontrar la relación entre el tiempo (índices numéricos) y los montos de ventas
model = LinearRegression()

X = numeric_time.reshape(-1, 1)  #  #Transforma de [1, 2, 3, 4, 5] a [[1], [2], [3], [4], [5]]

"""
 print(X)
 [ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]]
"""
y = monthly_sales.values.reshape(-1, 1)  # hacemos transformacion de  (1D, tamaño (n,)) a  (2D, tamaño (n, 1))
model.fit(X, y)
trend_line = model.predict(X)
plt.plot(monthly_sales.index, trend_line.flatten(), color='red', linestyle='--', linewidth=2, label='Trend Line')

plt.title('Monthly Transaction Amount by Year with Trend Line', fontsize=16)
plt.xlabel('Year-Month', fontsize=12)
plt.ylabel('Total Amount (€)', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.grid(axis='x', visible=False)  
ax = plt.gca()  # Obtener el objeto de los ejes
formatter = FuncFormatter(lambda x, pos: f'{int(x / 1000)}K')  # Dividir por 1000 y mostrar "K"
ax.yaxis.set_major_formatter(formatter)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout(pad=0, rect=[0.2, 0, 0, 0])  
plt.subplots_adjust(bottom=0.25)  
plt.show()


# Ej 5

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 

# dataset = pandas.DataFrame(country, declined_status)
# dataset = dataset.drop_duplicates()
merged_df=dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
sns.set_theme(style="white", palette='pastel')
plt.figure(figsize=(8, 5))
sns.barplot(merged_df, x="amount", y="country", estimator="count", hue="declined_status" , order = merged_df['country'].value_counts().index,errorbar=None)
plt.title('Transaction Distribution by Country and Status', fontsize=16)
plt.ylabel('Country')
plt.xlabel('Number of Transactions')
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.grid(axis='y', visible=False)         
plt.legend(title='Transaction Status', fontsize=12)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout(pad=0)
plt.show()


# Ej 6

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 

# dataset = pandas.DataFrame(age, amount, user_id)
# dataset = dataset.drop_duplicates()
merged_df=dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.ticker import FuncFormatter

# Create bins for age ranges (e.g., groups of 5 years)
age_bins = range(merged_df['age'].min() - (merged_df['age'].min() % 5), merged_df['age'].max() + 5, 5)
merged_df['age_bin'] = pd.cut(merged_df['age'], bins=age_bins)

# Group by age bins and calculate the metrics
#observed=False: no incluir categorías vacías, si hay grupos de edades sin transacciones o sin clientes
#observed=True: inclue todas las categorias aunque esten vacios y sin datos
grouped_data = merged_df.groupby('age_bin',observed=True).agg(
    total_amount=('amount', 'sum'),
    unique_clients=('user_id', 'nunique')  # 'nunique' to count unique values
).reset_index()

sns.set_theme(style="white", palette='pastel')
fig, ax1 = plt.subplots(figsize=(6.5, 4))

# Bar plot for total amount
ax1.bar(grouped_data['age_bin'].astype(str), grouped_data['total_amount'], alpha=0.7)
ax1.set_ylabel('Total Amount (€)')
formatter = FuncFormatter(lambda x, pos: f'{int(x / 1000)}K')  # Dividir por 1000 y mostrar "K"
ax1.yaxis.set_major_formatter(formatter)
#ax1.tick_params(axis='y', labelcolor='blue')
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.grid(axis='x', visible=False)         

# Line plot for unique clients
ax2 = ax1.twinx()
ax2.plot(grouped_data['age_bin'].astype(str), grouped_data['unique_clients'], color='darkgreen', marker='o', label='Unique Clients')
ax2.set_ylabel('Unique Clients', color='darkgreen')
ax2.tick_params(axis='y', labelcolor='darkgreen')

# Add title and legend
plt.title('Transaction Amount and Unique Clients by Age Ranges', fontsize=16)
ax1.set_xlabel('Age Bins')
fig.tight_layout()
plt.show()


# Ej 7

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 

# dataset = pandas.DataFrame(amount, year, month)
# dataset = dataset.drop_duplicates()
merged_df=dataset
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
plt.figure(figsize=(8,8))
sns.set(style="white")
sns.set_palette("pastel")
# Seleccionamos columnas relevantes para el análisis sin codificar texto innecesariamente
pairplot_df = merged_df[['amount', 'year', 'month']]
pairplot_df=pairplot_df.sort_values(by='month', ascending=True)
# Creamos el pairplot con 'hue' basado en texto directamente (países)
g = sns.pairplot(pairplot_df, hue='year', palette='pastel', diag_kind="hist",diag_kws=dict(fill=True, bins=12), plot_kws={'alpha': 0.9})
for ax in g.axes.flatten():
    if ax.get_xlabel() == 'month':  # Ajuste en el eje X
        ax.set_xticks(range(1, 13))  # Colocar los números de los meses (1-12)
        ax.set_xticklabels(range(1, 13))  # Etiquetas de los meses
        ax.set_xlim(0.5, 12.5)  # Ajustar los límites para que no se recorten
    if ax.get_ylabel() == 'month':  # Ajuste en el eje Y
        ax.set_yticks(range(1, 13))  # Colocar los números de los meses (1-12)
        ax.set_yticklabels(range(1, 13))  # Etiquetas de los meses
        ax.set_ylim(0.5, 12.5)  # Ajustar los límites para que no se recorten
g.fig.subplots_adjust(top=0.85)  # Reduce the top space of the figure to make room for the title
g.fig.suptitle('Distribution of Transaction Amounts by Month and Year', fontsize=16, y=0.92)

plt.show()

# NIVEL 2

# Ej 1

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 

# dataset = pandas.DataFrame(country, declined_status)
# dataset = dataset.drop_duplicates()
merged_df=dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
sns.set_theme(style="white", palette='pastel')
sns.set(style="white")
sns.set_palette("pastel")
numeric_cols = merged_df.select_dtypes(include=['float64', 'int64', 'int32'])
correlation_matrix = numeric_cols.corr()
plt.figure(figsize=(8, 6))
g=sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix of Numerical Variables', fontsize=16, pad=20)
plt.tight_layout()
plt.show()


# Ej 2

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 

# dataset = pandas.DataFrame(amount, year, month)
# dataset = dataset.drop_duplicates()
merged_df=dataset
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
plt.figure(figsize=(8,8))
sns.set(style="white")
sns.set_palette("pastel")
declined_user_id = merged_df.groupby(['user_id', 'declined']).agg(
    total_amount=('amount', 'sum'),
    count=('amount', 'size') 
).reset_index()
g = sns.jointplot(data=declined_user_id, x='user_id', y='count', hue='declined')
g.fig.suptitle('Relationship Between User ID and Transaction Count', fontsize=16)
g.fig.tight_layout()
plt.show()


# NIVEL 3

# Ej 1

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 
# dataset = pandas.DataFrame(country, declined_status)
# dataset = dataset.drop_duplicates()
merged_df=dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
sns.set_theme(style="white", palette='pastel')
sns.set(style="white")
sns.set_palette("pastel")
merged_df = merged_df.sort_values(by='country_user', ascending=True)
plt.figure(figsize=(5, 6))
sns.violinplot(data=merged_df, x='age', y='country_user', hue='country_user', palette='pastel', inner=None)
sns.stripplot(data=merged_df, x='age', y='country_user', color='darkgreen', alpha=0.3)
plt.title("Age Distribution by User's Country", fontsize=16, pad=20)
plt.xlabel('Age', labelpad=10)
plt.ylabel('')
plt.tight_layout(pad=2.0, rect=[0, 0, 1, 1])
plt.tight_layout(pad=0)
plt.show()



# Ej 2

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 
# dataset = pandas.DataFrame(amount, year, month)
# dataset = dataset.drop_duplicates()
merged_df=dataset
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
plt.figure(figsize=(8,8))
sns.set(style="white")
sns.set_palette("pastel")
monthly_sales_by_country_user = merged_df.groupby(['year_month', 'country_user','declined_status'])['amount'].sum().reset_index()
g = sns.FacetGrid(monthly_sales_by_country_user, col='declined_status', row='country_user', hue='country_user', height=2, aspect=2, margin_titles=True)
g.map(sns.barplot, 'year_month','amount')
g.set_titles(col_template="{col_name}", row_template="{row_name}")
for ax in g.axes.flatten():
    ax.set_xticks(range(len(monthly_sales_by_country_user['year_month'].unique())))  
    ax.set_xticklabels(monthly_sales_by_country_user['year_month'].unique(), rotation=90, ha='center') 
g.fig.subplots_adjust(hspace=1, top=0.9) 
g.set_axis_labels('', '')
g.fig.suptitle("Monthly Transaction Amounts (€) by User's Country and Decline Status", fontsize=16)
plt.show()
